# Fixed effect regression

In [1]:
import pandas as pd 
import numpy as np
from linearmodels.panel import PanelOLS
from linearmodels.panel import PooledOLS
import statsmodels.api as sm
import statsmodels.formula.api as smf

In [2]:
panel_data = pd.read_csv("panel_data.csv")
print(panel_data.head())

         date  isin  cs_change  delta lr  change_in_10y_bond  change_in_slope  \
0  2015-01-31     8   -0.00063  0.000000            -0.00395          0.00006   
1  2015-01-31    16    0.00008  0.001782            -0.00395          0.00006   
2  2015-01-31    19   -0.00046  0.000000            -0.00395          0.00006   
3  2015-01-31    25   -0.00077  0.000209            -0.00395          0.00006   
4  2015-01-31    31   -0.00078  0.000000            -0.00395          0.00006   

   smi_return  v3x_change rating   maturity  cpi_change  unempl_change  \
0   -0.068915    0.045699    AAA   8.000000     -0.4006            0.3   
1   -0.068915    0.045699     NR  10.000000     -0.4006            0.3   
2   -0.068915    0.045699     NR   8.000000     -0.4006            0.3   
3   -0.068915    0.045699     A-   6.833333     -0.4006            0.3   
4   -0.068915    0.045699     AA  10.000000     -0.4006            0.3   

   gdp_change  liquidity  credit_rating_numeric     rating_class  \


In [3]:
panel_data = panel_data.rename(columns={'delta lr':'change_in_lr'})

In [4]:
panel_data_adj = panel_data.set_index(['date', 'isin']).sort_index()
print(panel_data_adj.head())

                 cs_change  change_in_lr  change_in_10y_bond  change_in_slope  \
date       isin                                                                 
2015-01-31 8      -0.00063      0.000000            -0.00395          0.00006   
           16      0.00008      0.001782            -0.00395          0.00006   
           19     -0.00046      0.000000            -0.00395          0.00006   
           25     -0.00077      0.000209            -0.00395          0.00006   
           31     -0.00078      0.000000            -0.00395          0.00006   

                 smi_return  v3x_change rating   maturity  cpi_change  \
date       isin                                                         
2015-01-31 8      -0.068915    0.045699    AAA   8.000000     -0.4006   
           16     -0.068915    0.045699     NR  10.000000     -0.4006   
           19     -0.068915    0.045699     NR   8.000000     -0.4006   
           25     -0.068915    0.045699     A-   6.833333     -0.40

In [5]:
X = panel_data_adj[['change_in_lr', 'change_in_10y_bond', 'change_in_slope', 'smi_return', 'v3x_change', 'cpi_change', 'unempl_change', 'gdp_change', 'liquidity']]
y = panel_data_adj['cs_change']
X = sm.add_constant(X)

model = PanelOLS(y, X, time_effects=True).fit()
print(model.summary)

                          PanelOLS Estimation Summary                           
Dep. Variable:              cs_change   R-squared:                        0.0322
Estimator:                   PanelOLS   R-squared (Between):              0.0695
No. Observations:               36220   R-squared (Within):              -0.0034
Date:                Thu, May 15 2025   R-squared (Overall):              0.0297
Time:                        17:14:05   Log-likelihood                  1.79e+05
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      132.12
Entities:                         120   P-value                           0.0000
Avg Obs:                       301.83   Distribution:                 F(9,35683)
Min Obs:                       45.000                                           
Max Obs:                       528.00   F-statistic (robust):             132.12
                            

### Fixed effect regression by classes

In [6]:
results_by_rating = {}
r_squared_table = {}

coef_table = pd.DataFrame()
for group_name, group_data in panel_data_adj.groupby('rating_class'):
    y_group = group_data['cs_change']
    x_group = group_data[['change_in_lr', 'change_in_10y_bond', 'change_in_slope', 'smi_return', 'v3x_change', 'cpi_change', 'unempl_change', 'gdp_change', 'liquidity']]
    x_group = sm.add_constant(x_group)  
    model = PanelOLS(y_group, x_group, entity_effects=True, drop_absorbed=True).fit()
    
    coef = model.params
    coef.name = group_name
    coef_table = pd.concat([coef_table, coef], axis=1)
    
    
    print(f"\n\nResult for {group_name}")
    print(model.summary)
    
  
    results_by_rating[group_name] = model
    r_squared_table[group_name] = model.rsquared
    

coef_table = coef_table.T



Result for A class bonds
                          PanelOLS Estimation Summary                           
Dep. Variable:              cs_change   R-squared:                        0.0020
Estimator:                   PanelOLS   R-squared (Between):              0.0033
No. Observations:                4452   R-squared (Within):               0.0020
Date:                Thu, May 15 2025   R-squared (Overall):              0.0018
Time:                        17:14:05   Log-likelihood                 2.481e+04
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      4.3483
Entities:                         120   P-value                           0.0130
Avg Obs:                       37.100   Distribution:                  F(2,4330)
Min Obs:                       6.0000                                           
Max Obs:                       65.000   F-statistic (robust):             4.3483
 

/var/folders/dw/93tk385j54q9dmq3f082vbnm0000gn/T/ipykernel_94474/3221789431.py:9: AbsorbingEffectWarning: 
Variables have been fully absorbed and have removed from the regression:

change_in_10y_bond, change_in_slope, smi_return, v3x_change, cpi_change, unempl_change, gdp_change

  model = PanelOLS(y_group, x_group, entity_effects=True, drop_absorbed=True).fit()
/var/folders/dw/93tk385j54q9dmq3f082vbnm0000gn/T/ipykernel_94474/3221789431.py:9: AbsorbingEffectWarning: 
Variables have been fully absorbed and have removed from the regression:

change_in_10y_bond, change_in_slope, smi_return, v3x_change, cpi_change, unempl_change, gdp_change

  model = PanelOLS(y_group, x_group, entity_effects=True, drop_absorbed=True).fit()
/var/folders/dw/93tk385j54q9dmq3f082vbnm0000gn/T/ipykernel_94474/3221789431.py:9: AbsorbingEffectWarning: 
Variables have been fully absorbed and have removed from the regression:

change_in_10y_bond, change_in_slope, smi_return, v3x_change, cpi_change, unempl_change, g

In [7]:
print(coef_table)

                    const  change_in_lr  liquidity
A class bonds   -0.000078      0.000759   0.000076
AA class bonds  -0.000100      0.000836   0.000079
AAA class bonds -0.000049      0.000711   0.000032
BBB/BB/B bonds   0.000078     -0.001194  -0.000055
Not rated bonds -0.000026     -0.000341   0.000008


In [8]:
# Create R² DataFrame
r_squared_table = pd.Series(r_squared_table, name='R_squared')
# Print
print(r_squared_table)

A class bonds      0.002004
AA class bonds     0.002660
AAA class bonds    0.000220
BBB/BB/B bonds     0.000089
Not rated bonds    0.000008
Name: R_squared, dtype: float64


### By maturities

In [9]:
results_by_maturity = {}
r_squared_table = {}

coef_table = pd.DataFrame()
for group_name, group_data in panel_data_adj.groupby('maturity_bucket'):
    y_group = group_data['cs_change']
    x_group = group_data[['change_in_lr', 'change_in_10y_bond', 'change_in_slope', 'smi_return', 'v3x_change', 'cpi_change', 'unempl_change', 'gdp_change', 'liquidity']]
    x_group = sm.add_constant(x_group)  
    model = PanelOLS(y_group, x_group, entity_effects=True, drop_absorbed=True).fit()
    
    coef = model.params
    coef.name = group_name
    coef_table = pd.concat([coef_table, coef], axis=1)
    
    
    print(f"\n\nResult for {group_name}")
    print(model.summary)
    
  
    results_by_maturity[group_name] = model
    r_squared_table[group_name] = model.rsquared
    

coef_table = coef_table.T



Result for Long-Term
                          PanelOLS Estimation Summary                           
Dep. Variable:              cs_change   R-squared:                        0.0003
Estimator:                   PanelOLS   R-squared (Between):              0.0023
No. Observations:               16933   R-squared (Within):               0.0003
Date:                Thu, May 15 2025   R-squared (Overall):              0.0007
Time:                        17:14:05   Log-likelihood                 8.607e+04
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      2.3264
Entities:                         120   P-value                           0.0977
Avg Obs:                       141.11   Distribution:                 F(2,16811)
Min Obs:                       19.000                                           
Max Obs:                       259.00   F-statistic (robust):             2.3264
     

/var/folders/dw/93tk385j54q9dmq3f082vbnm0000gn/T/ipykernel_94474/2396249024.py:9: AbsorbingEffectWarning: 
Variables have been fully absorbed and have removed from the regression:

change_in_10y_bond, change_in_slope, smi_return, v3x_change, cpi_change, unempl_change, gdp_change

  model = PanelOLS(y_group, x_group, entity_effects=True, drop_absorbed=True).fit()
/var/folders/dw/93tk385j54q9dmq3f082vbnm0000gn/T/ipykernel_94474/2396249024.py:9: AbsorbingEffectWarning: 
Variables have been fully absorbed and have removed from the regression:

change_in_10y_bond, change_in_slope, smi_return, v3x_change, cpi_change, unempl_change, gdp_change

  model = PanelOLS(y_group, x_group, entity_effects=True, drop_absorbed=True).fit()
/var/folders/dw/93tk385j54q9dmq3f082vbnm0000gn/T/ipykernel_94474/2396249024.py:9: AbsorbingEffectWarning: 
Variables have been fully absorbed and have removed from the regression:

change_in_10y_bond, change_in_slope, smi_return, v3x_change, cpi_change, unempl_change, g

In [10]:
# Create R² DataFrame
r_squared_table = pd.Series(r_squared_table, name='R_squared')
# Print
print(r_squared_table)

Long-Term      0.000277
Medium-Term    0.000376
Short-Term     0.001476
Name: R_squared, dtype: float64


### Equity return testing

In [11]:
eq_data = pd.read_csv("equity_panel.csv")
print(eq_data.head())

   Unnamed: 0        date  isin  cs_change  delta lr  change_in_10y_bond  \
0           0  2022-11-30     0   -0.00267  0.000058            -0.00028   
1           1  2022-12-31     0   -0.00102  0.000058             0.00517   
2           2  2023-01-31     0   -0.00020  0.000024            -0.00310   
3           3  2023-02-28     0    0.00077  0.000024             0.00167   
4           4  2023-03-31     0    0.00259  0.000024            -0.00229   

   change_in_slope  smi_return  v3x_change rating  ...  cpi_change  \
0         -0.00395    0.027314   -0.030361    AAA  ...      0.0040   
1          0.00171   -0.036456   -0.005036    AAA  ...     -0.2283   
2         -0.00158    0.050557    0.005429    AAA  ...      0.6286   
3         -0.00069   -0.016747   -0.001126    AAA  ...      0.7838   
4         -0.00179    0.000710   -0.016202    AAA  ...      0.1623   

   unempl_change  gdp_change  liquidity  credit_rating_numeric  \
0            0.1    0.260500      0.665                 

In [12]:
eq_data = eq_data.drop(columns='Unnamed: 0')
print(eq_data.head())

         date  isin  cs_change  delta lr  change_in_10y_bond  change_in_slope  \
0  2022-11-30     0   -0.00267  0.000058            -0.00028         -0.00395   
1  2022-12-31     0   -0.00102  0.000058             0.00517          0.00171   
2  2023-01-31     0   -0.00020  0.000024            -0.00310         -0.00158   
3  2023-02-28     0    0.00077  0.000024             0.00167         -0.00069   
4  2023-03-31     0    0.00259  0.000024            -0.00229         -0.00179   

   smi_return  v3x_change rating  maturity  cpi_change  unempl_change  \
0    0.027314   -0.030361    AAA      10.0      0.0040            0.1   
1   -0.036456   -0.005036    AAA      10.0     -0.2283            0.1   
2    0.050557    0.005429    AAA      10.0      0.6286            0.1   
3   -0.016747   -0.001126    AAA      10.0      0.7838            0.0   
4    0.000710   -0.016202    AAA      10.0      0.1623           -0.1   

   gdp_change  liquidity  credit_rating_numeric     rating_class  \
0    0

In [13]:
eq_data_adj = eq_data.set_index(['date', 'isin']).sort_index()
print(eq_data_adj.head())

                 cs_change  delta lr  change_in_10y_bond  change_in_slope  \
date       isin                                                             
2015-01-31 8      -0.00063  0.000000            -0.00395          0.00006   
           16      0.00008  0.001782            -0.00395          0.00006   
           19     -0.00046  0.000000            -0.00395          0.00006   
           25     -0.00077  0.000209            -0.00395          0.00006   
           31     -0.00078  0.000000            -0.00395          0.00006   

                 smi_return  v3x_change rating   maturity  cpi_change  \
date       isin                                                         
2015-01-31 8      -0.068915    0.045699    AAA   8.000000     -0.4006   
           16     -0.068915    0.045699     NR  10.000000     -0.4006   
           19     -0.068915    0.045699     NR   8.000000     -0.4006   
           25     -0.068915    0.045699     A-   6.833333     -0.4006   
           31     -0.0

In [14]:
X = eq_data_adj[['delta lr', 'change_in_10y_bond', 'change_in_slope', 'smi_return', 'v3x_change', 'cpi_change', 'unempl_change', 'gdp_change', 'liquidity']]
y = eq_data_adj['cs_change']
X = sm.add_constant(X)

model = PanelOLS(y, X, entity_effects=True, drop_absorbed=True).fit()
print(model.summary)

                          PanelOLS Estimation Summary                           
Dep. Variable:              cs_change   R-squared:                        0.0007
Estimator:                   PanelOLS   R-squared (Between):              0.0058
No. Observations:               13668   R-squared (Within):               0.0007
Date:                Thu, May 15 2025   R-squared (Overall):              0.0014
Time:                        17:14:05   Log-likelihood                 6.883e+04
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      4.6438
Entities:                         120   P-value                           0.0096
Avg Obs:                       113.90   Distribution:                 F(2,13546)
Min Obs:                       18.000                                           
Max Obs:                       197.00   F-statistic (robust):             4.6438
                            

/var/folders/dw/93tk385j54q9dmq3f082vbnm0000gn/T/ipykernel_94474/2176528053.py:5: AbsorbingEffectWarning: 
Variables have been fully absorbed and have removed from the regression:

change_in_10y_bond, change_in_slope, smi_return, v3x_change, cpi_change, unempl_change, gdp_change

  model = PanelOLS(y, X, entity_effects=True, drop_absorbed=True).fit()


### By return classes

In [15]:
results_by_returns = {}
r_squared_table = {}

coef_table = pd.DataFrame()
for group_name, group_data in eq_data_adj.groupby('return_bucket'):
    y_group = group_data['cs_change']
    x_group = group_data[['delta lr', 'change_in_10y_bond', 'change_in_slope', 'smi_return', 'v3x_change', 'cpi_change', 'unempl_change', 'gdp_change', 'liquidity']]
    x_group = sm.add_constant(x_group)  
    model = PanelOLS(y_group, x_group, entity_effects=True, drop_absorbed=True).fit()
    
    coef = model.params
    coef.name = group_name
    coef_table = pd.concat([coef_table, coef], axis=1)
    
    
    print(f"\n\nResult for {group_name}")
    print(model.summary)
    
  
    results_by_returns[group_name] = model
    r_squared_table[group_name] = model.rsquared
    

coef_table = coef_table.T



Result for High
                          PanelOLS Estimation Summary                           
Dep. Variable:              cs_change   R-squared:                        0.0008
Estimator:                   PanelOLS   R-squared (Between):             -0.0157
No. Observations:                3987   R-squared (Within):               0.0008
Date:                Thu, May 15 2025   R-squared (Overall):              0.0009
Time:                        17:14:05   Log-likelihood                  2.23e+04
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      1.6328
Entities:                         119   P-value                           0.1955
Avg Obs:                       33.504   Distribution:                  F(2,3866)
Min Obs:                       1.0000                                           
Max Obs:                       127.00   F-statistic (robust):             1.6328
          

/var/folders/dw/93tk385j54q9dmq3f082vbnm0000gn/T/ipykernel_94474/3048476849.py:9: AbsorbingEffectWarning: 
Variables have been fully absorbed and have removed from the regression:

change_in_10y_bond, change_in_slope, smi_return, v3x_change, cpi_change, unempl_change, gdp_change

  model = PanelOLS(y_group, x_group, entity_effects=True, drop_absorbed=True).fit()
/var/folders/dw/93tk385j54q9dmq3f082vbnm0000gn/T/ipykernel_94474/3048476849.py:9: AbsorbingEffectWarning: 
Variables have been fully absorbed and have removed from the regression:

change_in_10y_bond, change_in_slope, smi_return, v3x_change, cpi_change, unempl_change, gdp_change

  model = PanelOLS(y_group, x_group, entity_effects=True, drop_absorbed=True).fit()
/var/folders/dw/93tk385j54q9dmq3f082vbnm0000gn/T/ipykernel_94474/3048476849.py:9: AbsorbingEffectWarning: 
Variables have been fully absorbed and have removed from the regression:

change_in_10y_bond, change_in_slope, smi_return, v3x_change, cpi_change, unempl_change, g

In [16]:
# Create R² DataFrame
r_squared_table = pd.Series(r_squared_table, name='R_squared')
# Print
print(r_squared_table)

High      0.000844
Low       0.021713
Medium    0.000069
Name: R_squared, dtype: float64


## Equity testing with market leverage

In [17]:
eq_data_2 = pd.read_csv("equity_panel2.csv")
print(eq_data_2.head())

   Unnamed: 0        date  isin  cs_change  delta lr  change_in_10y_bond  \
0           0  2022-11-30     0   -0.00267  0.000058            -0.00028   
1           1  2022-12-31     0   -0.00102  0.000058             0.00517   
2           2  2023-01-31     0   -0.00020  0.000024            -0.00310   
3           3  2023-02-28     0    0.00077  0.000024             0.00167   
4           4  2023-03-31     0    0.00259  0.000024            -0.00229   

   change_in_slope  smi_return  v3x_change rating  ...  unempl_change  \
0         -0.00395    0.027314   -0.030361    AAA  ...            0.1   
1          0.00171   -0.036456   -0.005036    AAA  ...            0.1   
2         -0.00158    0.050557    0.005429    AAA  ...            0.1   
3         -0.00069   -0.016747   -0.001126    AAA  ...            0.0   
4         -0.00179    0.000710   -0.016202    AAA  ...           -0.1   

   gdp_change  liquidity  credit_rating_numeric     rating_class  \
0    0.260500      0.665            

In [18]:
eq_data_2 = eq_data_2.drop(columns='Unnamed: 0')
print(eq_data_2.head())

         date  isin  cs_change  delta lr  change_in_10y_bond  change_in_slope  \
0  2022-11-30     0   -0.00267  0.000058            -0.00028         -0.00395   
1  2022-12-31     0   -0.00102  0.000058             0.00517          0.00171   
2  2023-01-31     0   -0.00020  0.000024            -0.00310         -0.00158   
3  2023-02-28     0    0.00077  0.000024             0.00167         -0.00069   
4  2023-03-31     0    0.00259  0.000024            -0.00229         -0.00179   

   smi_return  v3x_change rating  maturity  ...  unempl_change  gdp_change  \
0    0.027314   -0.030361    AAA      10.0  ...            0.1    0.260500   
1   -0.036456   -0.005036    AAA      10.0  ...            0.1    0.260500   
2    0.050557    0.005429    AAA      10.0  ...            0.1    0.061917   
3   -0.016747   -0.001126    AAA      10.0  ...            0.0    0.061917   
4    0.000710   -0.016202    AAA      10.0  ...           -0.1    0.061917   

   liquidity  credit_rating_numeric     rati

In [19]:
eq_data_2_adj = eq_data_2.set_index(['date', 'isin']).sort_index()
print(eq_data_2_adj.head())

                 cs_change  delta lr  change_in_10y_bond  change_in_slope  \
date       isin                                                             
2015-01-31 8      -0.00063  0.000000            -0.00395          0.00006   
           16      0.00008  0.001782            -0.00395          0.00006   
           19     -0.00046  0.000000            -0.00395          0.00006   
           25     -0.00077  0.000209            -0.00395          0.00006   
           56      0.00328  0.000000            -0.00395          0.00006   

                 smi_return  v3x_change rating   maturity  cpi_change  \
date       isin                                                         
2015-01-31 8      -0.068915    0.045699    AAA   8.000000     -0.4006   
           16     -0.068915    0.045699     NR  10.000000     -0.4006   
           19     -0.068915    0.045699     NR   8.000000     -0.4006   
           25     -0.068915    0.045699     A-   6.833333     -0.4006   
           56     -0.0

In [20]:
X = eq_data_2_adj[['change_in_lmarket', 'change_in_10y_bond', 'change_in_slope', 'smi_return', 'v3x_change', 'cpi_change', 'unempl_change', 'gdp_change', 'liquidity']]
y = eq_data_2_adj['cs_change']
X = sm.add_constant(X)

model = PanelOLS(y, X, entity_effects=True, drop_absorbed=True).fit()
print(model.summary)

                          PanelOLS Estimation Summary                           
Dep. Variable:              cs_change   R-squared:                        0.0006
Estimator:                   PanelOLS   R-squared (Between):              0.0055
No. Observations:               12992   R-squared (Within):               0.0006
Date:                Thu, May 15 2025   R-squared (Overall):              0.0013
Time:                        17:14:05   Log-likelihood                 6.516e+04
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      4.1117
Entities:                         120   P-value                           0.0164
Avg Obs:                       108.27   Distribution:                 F(2,12870)
Min Obs:                       16.000                                           
Max Obs:                       197.00   F-statistic (robust):             4.1117
                            

/var/folders/dw/93tk385j54q9dmq3f082vbnm0000gn/T/ipykernel_94474/3365734019.py:5: AbsorbingEffectWarning: 
Variables have been fully absorbed and have removed from the regression:

change_in_10y_bond, change_in_slope, smi_return, v3x_change, cpi_change, unempl_change, gdp_change

  model = PanelOLS(y, X, entity_effects=True, drop_absorbed=True).fit()


In [21]:
results_by_returns_2 = {}
r_squared_table = {}

coef_table = pd.DataFrame()
for group_name, group_data in eq_data_2_adj.groupby('return_bucket'):
    y_group = group_data['cs_change']
    x_group = group_data[['change_in_lmarket', 'change_in_10y_bond', 'change_in_slope', 'smi_return', 'v3x_change', 'cpi_change', 'unempl_change', 'gdp_change', 'liquidity']]
    x_group = sm.add_constant(x_group)  
    model = PanelOLS(y_group, x_group, entity_effects=True, drop_absorbed=True).fit()
    
    coef = model.params
    coef.name = group_name
    coef_table = pd.concat([coef_table, coef], axis=1)
    
    
    print(f"\n\nResult for {group_name}")
    print(model.summary)
    
  
    results_by_returns_2[group_name] = model
    r_squared_table[group_name] = model.rsquared
    

coef_table = coef_table.T



Result for High
                          PanelOLS Estimation Summary                           
Dep. Variable:              cs_change   R-squared:                        0.0004
Estimator:                   PanelOLS   R-squared (Between):             -0.0119
No. Observations:                3936   R-squared (Within):               0.0004
Date:                Thu, May 15 2025   R-squared (Overall):              0.0010
Time:                        17:14:05   Log-likelihood                 2.204e+04
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      0.7702
Entities:                         119   P-value                           0.4630
Avg Obs:                       33.076   Distribution:                  F(2,3815)
Min Obs:                       1.0000                                           
Max Obs:                       127.00   F-statistic (robust):             0.7702
          

/var/folders/dw/93tk385j54q9dmq3f082vbnm0000gn/T/ipykernel_94474/2946626842.py:9: AbsorbingEffectWarning: 
Variables have been fully absorbed and have removed from the regression:

change_in_10y_bond, change_in_slope, smi_return, v3x_change, cpi_change, unempl_change, gdp_change

  model = PanelOLS(y_group, x_group, entity_effects=True, drop_absorbed=True).fit()
/var/folders/dw/93tk385j54q9dmq3f082vbnm0000gn/T/ipykernel_94474/2946626842.py:9: AbsorbingEffectWarning: 
Variables have been fully absorbed and have removed from the regression:

change_in_10y_bond, change_in_slope, smi_return, v3x_change, cpi_change, unempl_change, gdp_change

  model = PanelOLS(y_group, x_group, entity_effects=True, drop_absorbed=True).fit()
/var/folders/dw/93tk385j54q9dmq3f082vbnm0000gn/T/ipykernel_94474/2946626842.py:9: AbsorbingEffectWarning: 
Variables have been fully absorbed and have removed from the regression:

change_in_10y_bond, change_in_slope, smi_return, v3x_change, cpi_change, unempl_change, g

In [22]:
# Create R² DataFrame
r_squared_table = pd.Series(r_squared_table, name='R_squared')
# Print
print(r_squared_table)

High      0.000404
Low       0.018857
Medium    0.000075
Name: R_squared, dtype: float64


In [23]:
eq_data_2_adj.to_csv("panel2_adj.csv")